In [2]:
from pathlib import Path

import pandas as pd
from catboost import CatBoostClassifier


def load_data(data_path: Path):
    df_train = pd.read_csv(data_path / "train.csv")
    df_valid = pd.read_csv(data_path / "val.csv")
    df_test = pd.read_csv(data_path / "test.csv")

    X_train = df_train.drop(columns=["label"])
    y_train = df_train["label"]
    X_valid = df_valid.drop(columns=["label"])
    y_valid = df_valid["label"]
    X_test = df_test.drop(columns=["label"])
    y_test = df_test["label"]

    return X_train, y_train, X_valid, y_valid, X_test, y_test


def load_model(model_path: Path):
    try:
        model = CatBoostClassifier()
        model.load_model(model_path)
        return model
    except Exception:
        pass

    try:
        model = xgb.XGBClassifier()
        model.load_model(model_path)
        return model
    except Exception:
        pass

    try:
        model = lgb.Booster(model_file=model_path)
        return model
    except Exception:
        pass

    return None

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef


def evaluate_model(clf, X_test, y_test, metric):
    y_pred = clf.predict(X_test)
    if y_pred.ndim == 1:
        y_pred = (y_pred > 0.5).astype(int)
    else:
        y_pred = np.argmax(y_pred, axis=1)

    if metric == "F1":
        return f1_score(y_test, y_pred, average="binary")
    elif metric == "Accuracy":
        return accuracy_score(y_test, y_pred)
    else:
        return matthews_corrcoef(y_test, y_pred)

In [4]:
embeddings_dir = Path("../data/embeddings")

DATASETS = {
    "enhancers": embeddings_dir / "enhancers",
    "promoter_all": embeddings_dir / "promoter_all",
    "splice_sites_all": embeddings_dir / "splice_sites_all",
    "H3K9me3": embeddings_dir / "H3K9me3",
    "H4K20me1": embeddings_dir / "H4K20me1",
}

METRICS = {
    "enhancers": "MCC",
    "promoter_all": "F1",
    "splice_sites_all": "Accuracy",
    "H3K9me3": "MCC",
    "H4K20me1": "MCC",
}

save_model_dir = Path("../two_diff_models")

In [9]:
import os


def evaluate_models(model_version):
    results = {}
    for dataset_name in DATASETS:
        results[dataset_name] = {}
        dataset_path = DATASETS[dataset_name]
        metric = METRICS[dataset_name]
        _, _, _, _, X_test, y_test = load_data(dataset_path)
        models_dir = save_model_dir / model_version
        
        model_path = models_dir / f"{dataset_name}_{model_version}.pkl"
        model = load_model(model_path)
        print(f"Evaluating {model_version} on {dataset_name} with {metric} metric")
        score = evaluate_model(model, X_test, y_test, metric)
        results[dataset_name][model_version] = score
    return results

In [10]:
catboost_v1_results = evaluate_models("catboost_v1")
catboost_v2_results = evaluate_models("catboost_v2")

Evaluating catboost_v1 on enhancers with MCC metric
Evaluating catboost_v1 on promoter_all with F1 metric
Evaluating catboost_v1 on splice_sites_all with Accuracy metric
Evaluating catboost_v1 on H3K9me3 with MCC metric
Evaluating catboost_v1 on H4K20me1 with MCC metric
Evaluating catboost_v2 on enhancers with MCC metric
Evaluating catboost_v2 on promoter_all with F1 metric
Evaluating catboost_v2 on splice_sites_all with Accuracy metric
Evaluating catboost_v2 on H3K9me3 with MCC metric
Evaluating catboost_v2 on H4K20me1 with MCC metric


In [22]:
catboost_v1_results

{'enhancers': {'catboost_v1': 0.48296352881712296},
 'promoter_all': {'catboost_v1': 0.8578745198463509},
 'splice_sites_all': {'catboost_v1': 0.343},
 'H3K9me3': {'catboost_v1': 0.306188482563984},
 'H4K20me1': {'catboost_v1': 0.5841163156296433}}

In [26]:
headers = ["Dataset", "catboost_v1", "catboost_v2"]
print(f"{headers[0]:<20} {headers[1]:>12} {headers[2]:>12}")
print("-" * 44)

for dataset in DATASETS:
    print(f"{dataset:<21} {catboost_v1_results[dataset]['catboost_v1']:.4f} {catboost_v2_results[dataset]['catboost_v2']:>12.4f}")

Dataset               catboost_v1  catboost_v2
--------------------------------------------
enhancers             0.4830       0.4700
promoter_all          0.8579       0.8542
splice_sites_all      0.3430       0.3430
H3K9me3               0.3062       0.3012
H4K20me1              0.5841       0.5718
